In [44]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from Layers.Embedding import Embedding
from Layers.Dense import DenseLayer
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from lstm import LSTMScratch

In [45]:
# Database already separated, so kind
train_df = pd.read_csv("../../dataset/rnn/train.csv")
val_df = pd.read_csv("../../dataset/rnn/valid.csv")
test_df = pd.read_csv("../../dataset/rnn/test.csv")

In [46]:
label_mapping = {"negative": 0,
                 "neutral": 1,
                 "positive": 2
                 }

# Take text and label only
train_df = train_df[["text", "label"]]
val_df = val_df[["text", "label"]]
test_df = test_df[["text", "label"]]

# Warning!!! Not idempotent operations :(
train_df["label"] = train_df["label"].map(label_mapping)
val_df["label"] = val_df["label"].map(label_mapping)
test_df["label"] = test_df["label"].map(label_mapping)

In [47]:
train_texts = train_df["text"].tolist()
train_labels = train_df["label"].tolist()

val_texts = val_df["text"].tolist()
val_labels = val_df["label"].tolist()

test_texts = test_df["text"].tolist()
test_labels = test_df["label"].tolist()

In [48]:
vectorizer = keras.layers.TextVectorization(
    output_mode='int'
)
vectorizer.adapt(train_texts)

In [49]:
x_train = vectorizer(np.array(train_texts))
x_val = vectorizer(np.array(val_texts))
x_test = vectorizer(np.array(test_texts))

y_train = np.array(train_labels)
y_val = np.array(val_labels)
y_test = np.array(test_labels)

In [50]:
# Scratch Embedder (Layers.Embedding)
embedder = Embedding(
    input_dim=len(vectorizer.get_vocabulary()), 
    output_dim=100
)

In [51]:
x_train_embed = embedder(x_train) 
x_val_embed = embedder(x_val) 
x_test_embed = embedder(x_test) 

In [52]:
print(x_train_embed.shape)

(500, 77, 100)


In [53]:
from sklearn.metrics import f1_score
from keras.callbacks import Callback
import numpy as np

class F1ScoreCallback(Callback):
    def __init__(self, X_val, y_val):
        self.X_val = X_val
        self.y_val = y_val

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.X_val)
        
        # Check if multi-class
        if y_pred.ndim > 1 and y_pred.shape[1] > 1:
            y_pred_labels = np.argmax(y_pred, axis=1)
        else:
            y_pred_labels = (y_pred > 0.5).astype(int).flatten()

        y_true_labels = self.y_val if len(self.y_val.shape) == 1 else np.argmax(self.y_val, axis=1)
        
        f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')
        print(f'Epoch {epoch + 1} - F1 Score: {f1:.4f}\n')

f1_callback = F1ScoreCallback(x_val_embed, y_val)

In [54]:
# model1 = keras.models.Sequential([
#     keras.layers.LSTM(256, return_sequences=True, input_shape=(x_train_embed.shape[1], x_train_embed.shape[2])),
#     keras.layers.LSTM(256),
    
#     keras.layers.Dense(3, activation="softmax")
# ])

# model1.compile(
#     loss='sparse_categorical_crossentropy',
#     optimizer="adam",
#     metrics=['accuracy']
# )

# model1.load_weights("./training_result/model1.weights.h5")

In [55]:
#     # keras.layers.LSTM(256, return_sequences=True, input_shape=(x_train_embed.shape[1], x_train_embed.shape[2])),
#     # keras.layers.LSTM(256),
    
#     # keras.layers.Dense(3, activation="softmax")


# model1_lstm1 = LSTMScratch(256, model1.layers[0].get_weights())
# model1_lstm2 = LSTMScratch(256, model1.layers[1].get_weights())
# model1_dense = DenseLayer(W=model1.layers[2].get_weights()[0], b=model1.layers[2].get_weights()[1], activation="softmax")

In [56]:
# library_result1 = model1.predict(x_test_embed)

In [57]:
# scratch_result1 = model1_lstm1.forward(x_test_embed)
# scratch_result1 = model1_lstm2.forward(scratch_result1)
# scratch_result1 = model1_dense.forward(scratch_result1)

In [58]:
# scratch_result1

In [59]:
# library_result1

In [60]:
library_model = keras.models.Sequential([
    keras.layers.LSTM(32, return_sequences=True, input_shape=(x_train_embed.shape[1], x_train_embed.shape[2])),
    keras.layers.LSTM(256, return_sequences=True),
    keras.layers.LSTM(64, return_sequences=True),
    keras.layers.LSTM(32),
    
    keras.layers.Dense(3, activation="softmax")
])

library_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer="adam",
    metrics=['accuracy']
)

library_model.load_weights("./training_result/model5.weights.h5")

c:\Users\SDN 214\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\SDN 214\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [61]:
scratch_model_lstm1 = LSTMScratch(32, library_model.layers[0].get_weights(), return_sequences=True)
scratch_model_lstm2 = LSTMScratch(256, library_model.layers[1].get_weights(), return_sequences=True)
scratch_model_lstm3 = LSTMScratch(64, library_model.layers[2].get_weights(), return_sequences=True)
scratch_model_lstm4 = LSTMScratch(32, library_model.layers[3].get_weights())
scratch_model_dense = DenseLayer(W=library_model.layers[4].get_weights()[0], b=library_model.layers[4].get_weights()[1], activation="softmax")

In [62]:
def predict_scratch(input_features):
    result = scratch_model_lstm1.forward(input_features)
    result = scratch_model_lstm2.forward(result)
    result = scratch_model_lstm3.forward(result)
    result = scratch_model_lstm4.forward(result)
    result = scratch_model_dense.forward(result)
    return result

In [63]:
library_result = library_model.predict(x_test_embed)

13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 186ms/step


In [64]:
scratch_result = predict_scratch(x_test_embed)

In [65]:
library_result

array([[0.38329223, 0.2144004 , 0.40230736],
       [0.38328892, 0.2144478 , 0.40226334],
       [0.3832975 , 0.21436313, 0.40233934],
       ...,
       [0.38318557, 0.2135591 , 0.40325525],
       [0.38329357, 0.21441635, 0.40229002],
       [0.38330147, 0.2142727 , 0.40242586]], dtype=float32)

In [66]:
scratch_result

array([[0.38329223, 0.21440042, 0.40230735],
       [0.38328891, 0.21444775, 0.40226334],
       [0.38329748, 0.21436317, 0.40233935],
       ...,
       [0.38318556, 0.21355915, 0.40325529],
       [0.38329359, 0.21441635, 0.40229006],
       [0.38330142, 0.21427269, 0.40242588]])

In [ ]:
library_predicted_class = np.argmax(library_result, axis=1) 
scratch_predicted_class = np.argmax(scratch_result, axis=1) 